In [1]:
import pandas as pd
import numpy as np
import ast

In [29]:
columns1 = ['genres', 'id', 'release_date']
columns = ['item_id',  "user_id",'playtime_forever']


In [30]:
df_generos = pd.read_csv("steam_games.csv", usecols=columns1)
df_tiempo = pd.read_csv("game_items.csv", usecols=columns)

In [38]:
#tiempo = df_tiempo.groupby(['item_id', 'user_id',])['playtime_forever'].agg('sum').reset_index()
tiempo = df_tiempo.groupby(['item_id', 'user_id'])['playtime_forever'].first().reset_index()

In [39]:
tiempo

,item_id,user_id,playtime_forever
0,10,-GM-Dragon,14
1,10,-KillZone-,987
2,10,-Mad-,8
3,10,-SEVEN-,617
4,10,-SatansLittleHelper-,511
...,...,...,...
5094077,529670,chidvd,0
5094078,529670,fearlesskeeper,0
5094079,529820,robinwalker,0
5094080,530720,CSMisBeast,0


In [5]:
tiempo

,item_id,user_id,playtime_forever
0,10,-GM-Dragon,14
1,10,-KillZone-,987
2,10,-Mad-,8
3,10,-SEVEN-,617
4,10,-SatansLittleHelper-,511
...,...,...,...
5094077,529670,chidvd,0
5094078,529670,fearlesskeeper,0
5094079,529820,robinwalker,0
5094080,530720,CSMisBeast,0


In [37]:
tiempo

,item_id,playtime_forever,user_id
0,10,17386015,76561197970982479
1,20,961702,76561197970982479
2,30,758991,76561197970982479
3,40,154486,76561197970982479
4,50,734562,76561197970982479
...,...,...,...
10973,528580,0,CSMisBeast
10974,528660,182,SirexPower
10975,529670,0,chidvd
10976,529820,0,robinwalker


In [40]:
df_generos.rename(columns={'id':'item_id'},inplace=True)
# Hacer merge (left join) con el df de user_items
df_merge = tiempo.merge(df_generos,on='item_id',how='left')
# Filtrar por los juegos que tengan playtime mayor a cero 0
df_merge = df_merge[df_merge['playtime_forever'] != 0]
# Eliminar nulos de la columna género
df_merge = df_merge.dropna(subset=['genres'])

In [41]:
df_merge

,item_id,user_id,playtime_forever,genres,release_date
0,10,-GM-Dragon,14,['Action'],2000-11-01
1,10,-KillZone-,987,['Action'],2000-11-01
2,10,-Mad-,8,['Action'],2000-11-01
3,10,-SEVEN-,617,['Action'],2000-11-01
4,10,-SatansLittleHelper-,511,['Action'],2000-11-01
...,...,...,...,...,...
5094040,527510,mayshowganmore,154,"['Casual', 'Indie', 'Simulation', 'Strategy']",2016-09-16
5094059,527810,lexby,2,"['Action', 'Indie']",2016-09-08
5094065,527890,SmokaBluntos,1,"['Casual', 'Simulation']",2016-09-12
5094066,527900,76561198159037032,44,"['Adventure', 'Casual']",2016-09-14


In [42]:
# Función para convertir a lista la columna de género
def tolist(lst):
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst =  ast.literal_eval(lst)
        # try:
        #     lst =  ast.literal_eval(lst)
        # except (SyntaxError, ValueError):
        #     return lst.lower()

    return list(lst)

In [43]:
df_merge['genres'] = df_merge['genres'].apply(lambda x : tolist(x))


In [44]:
df_merge['release_date'] = pd.to_datetime(df_merge['release_date'], errors='coerce')

# Identifica las filas con fechas que no se pudieron convertir.
fechas_invalidas = df_merge['release_date'].isna()

# Modifica las fechas invalidas (mes y año) para tomar el primer día del mes.
df_merge.loc[fechas_invalidas, 'release_date'] = df_merge[fechas_invalidas]['release_date'].apply(lambda x: x.replace(day=1))

In [45]:
df = df_merge.explode('genres')

# Agrupa los datos por año ('release_date') y género ('genres') y suma las horas jugadas.
grouped = df.groupby(['release_date', 'genres','user_id'])['playtime_forever'].first().reset_index()

# Modifica la agregación para sumar las horas de juego en lugar de eliminar duplicados.
grouped = grouped.groupby(['release_date', 'genres','user_id'])['playtime_forever'].first().reset_index()

In [46]:
grouped

,release_date,genres,user_id,playtime_forever
0,1983-06-19,Action,2Ta4,18
1,1983-06-19,Action,76561197966936422,331
2,1983-06-19,Action,76561197968887720,1
3,1983-06-19,Action,76561197969020980,98
4,1983-06-19,Action,76561197971401137,33
...,...,...,...,...
6669352,2018-12-01,Action,Cydran,21
6669353,2018-12-01,Adventure,Cydran,21
6669354,2018-12-01,Early Access,Cydran,21
6669355,2018-12-01,Indie,Cydran,21


In [47]:
grouped['release_date'] = pd.to_datetime(grouped['release_date'])

# Luego, crea una nueva columna 'year' para almacenar el año de lanzamiento.
grouped['year'] = grouped['release_date'].dt.year

# Ahora, agrupa los datos por 'year' y 'genres', y suma las horas jugadas ('playtime_forever') para cada combinación.
#resultados = grouped.groupby(['year', 'genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Finalmente, puedes iterar sobre los resultados para mostrar la cantidad de horas por género y año.
#for index, row in resultados.iterrows():
    #print(f"Año {row['year']} usuario {row['user_id']} género {row['genres']} playtime_forever: {row['playtime_forever']}")

In [48]:
# Primero, agrupa los datos por 'year', 'genres' y 'user_id' y suma las horas jugadas
resultado = grouped.groupby(['year', 'genres', 'user_id'])['playtime_forever'].first().reset_index()

# Luego, encuentra el usuario con el máximo playtime en cada género para cada año
#max_playtime_users = resultado.groupby(['year', 'genres', 'user_id'])['playtime_forever'].idxmax()

# Filtra el DataFrame original usando los índices encontrados anteriormente
#top_users_by_genre = resultado.loc[max_playtime_users]


In [18]:

# Primero, agrupa los datos por 'genres' y 'user_id' y suma las horas jugadas
grouped = resultado.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Luego, encuentra el usuario con el máximo playtime en cada género
max_playtime_users = grouped.groupby('genres')['playtime_forever'].idxmax()

# Filtra el DataFrame original usando los índices encontrados anteriormente
top_users_by_genre = grouped.loc[max_playtime_users]

# Ahora 'top_users_by_genre' contiene los usuarios con el máximo playtime en cada género de todos los años juntos

# Luego, agrupa 'resultados' por 'year', 'genres' y 'user_id' y suma las horas jugadas
grouped_by_year = resultado.groupby(['year', 'genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Filtra 'grouped_by_year' para incluir solo a los usuarios con el máximo playtime por género
filtered_grouped_by_year = grouped_by_year[grouped_by_year['user_id'].isin(top_users_by_genre['user_id'])]


In [20]:
filtered_grouped_by_year.to_csv('funciondos.csv', index=False)

In [49]:
grouped_by_genre = filtered_grouped_by_year.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Encuentra al usuario con el máximo playtime para cada género
max_playtime_users_by_genre = grouped_by_genre.groupby('genres')['playtime_forever'].idxmax()

# Filtra el DataFrame original 'grouped_by_genre' usando los índices encontrados anteriormente
top_users_by_genre = grouped_by_genre.loc[max_playtime_users_by_genre]


In [50]:
top_users_by_genre

,genres,user_id,playtime_forever
10,Action,Sp3ctre,1692650
21,Adventure,REBAS_AS_F-T,2182066
30,Animation &amp; Modeling,ScottyG555,168314
33,Audio Production,Lickidactyl,109916
39,Casual,REBAS_AS_F-T,1224933
50,Design &amp; Illustration,ScottyG555,168314
54,Early Access,76561197978756659,316969
67,Education,76561198059330972,65427
79,Free to Play,idonothack,808241
88,Indie,REBAS_AS_F-T,2401378


In [28]:
top_users_by_genre.to_csv('funciondos.csv', index=False)

In [25]:
# Primero, agrupa los datos por 'user_id', 'genres' y suma las horas jugadas
grouped_by_user_genre = filtered_grouped_by_year.groupby(['user_id', 'genres'])['playtime_forever'].sum().reset_index()

# Luego, encuentra el género en el que cada usuario tiene el máximo playtime acumulado
max_genre_by_user = grouped_by_user_genre.groupby('user_id')['playtime_forever'].idxmax()

# Filtra el DataFrame original 'grouped_by_user_genre' para obtener el género con el máximo playtime por usuario
top_genre_by_user = grouped_by_user_genre.loc[max_genre_by_user]

# Ahora 'top_genre_by_user' contiene el género con el máximo playtime acumulado por usuario

# Luego, agrupa los datos por 'user_id' y 'year' y suma las horas jugadas solo para el género con el máximo playtime
playtime_by_user_and_year = filtered_grouped_by_year.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

# Filtra el DataFrame 'playtime_by_user_and_year' para incluir solo los usuarios y años donde tienen el género con el máximo playtime
playtime_by_top_genre = playtime_by_user_and_year[playtime_by_user_and_year['user_id'].isin(top_genre_by_user['user_id'])]



In [27]:
playtime_by_top_genre.to_csv('funciondos1.csv', index=False)

In [31]:
max_playtime_users_by_genre = filtered_grouped_by_year.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()
max_playtime_users_by_genre = max_playtime_users_by_genre.groupby('genres').apply(lambda x: x.nlargest(1, 'playtime_forever')).reset_index(drop=True)

# Luego, filtra 'filtered_grouped_by_year' para incluir solo a los usuarios con el máximo playtime por género
filtered_grouped_by_genre_and_user = filtered_grouped_by_year.merge(max_playtime_users_by_genre, on=['genres', 'user_id'], how='inner')

# Finalmente, agrupa los datos filtrados por 'user_id' y 'genres' y suma las horas jugadas para cada usuario
user_playtime_by_genre = filtered_grouped_by_genre_and_user.groupby(['user_id', 'genres'])['playtime_forever'].sum().reset_index()

KeyError: 'Column not found: playtime_forever'

,year,genres,user_id,playtime_forever
0,1988,Action,DownSyndromeKid,664
2,1988,Adventure,DownSyndromeKid,664
5,1988,Indie,Lickidactyl,104
6,1988,RPG,Lickidactyl,104
7,1988,Strategy,Lickidactyl,104
...,...,...,...,...
1336,2017,Strategy,REBAS_AS_F-T,21137
1341,2018,Action,DownSyndromeKid,173
1342,2018,Early Access,DownSyndromeKid,173
1343,2018,Indie,DownSyndromeKid,173
